In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def load_data(file_path):
    """Load data from CSV file."""
    return pd.read_csv(file_path)

def plot_bars_with_error(ax, x, data_dict, colors, bar_width=0.2):
    """Plot grouped bar chart with error bars and return positions and means."""
    group_positions = {}
    group_means = {}

    for i, (group_name, cols) in enumerate(data_dict.items()):
        group_data = df[cols]
        means = group_data.mean(axis=1)
        sems = group_data.sem(axis=1)

        # Calculate bar positions (centered)
        positions = x + (i - (len(data_dict)-1)/2) * bar_width

        # Plot bars and error bars
        ax.bar(positions, means, width=bar_width, color=colors[group_name], label=group_name, capsize=4)
        ax.errorbar(positions, means, yerr=sems, fmt='none', ecolor=colors[group_name], capsize=4, elinewidth=2)

        group_positions[group_name] = positions
        group_means[group_name] = means

    return group_positions, group_means

def format_tick_labels(values):
    """Format x-axis tick labels by removing trailing zeros."""
    def fmt(x):
        return str(int(x)) if float(x).is_integer() else str(x)
    return [fmt(val) for val in values]

def get_significance_stars(p_value):
    """Return significance stars based on p-value."""
    if p_value < 0.0001:
        return '****'
    elif p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < 0.05:
        return '*'
    return None

def perform_anova_and_tukey(data, labels):
    """Perform one-way ANOVA and Tukey HSD test."""
    anova_result = stats.f_oneway(*[data[labels == label] for label in np.unique(labels)])
    tukey_result = pairwise_tukeyhsd(data, labels)
    return anova_result, tukey_result

def annotate_significance(ax, group_positions, group_means, tukey_result, pairs, idx):
    """Add significance annotations on the plot for given pairs of groups."""
    for g1, g2 in pairs:
        for res in tukey_result.summary().data[1:]:
            tg1, tg2, _, p_adj, _, _, reject = res
            if (tg1 == g1 and tg2 == g2) or (tg1 == g2 and tg2 == g1):
                star = get_significance_stars(p_adj)
                if star:
                    pos1 = group_positions[g1][idx]
                    pos2 = group_positions[g2][idx]
                    max_height = max(group_means[g1][idx], group_means[g2][idx])
                    height = max_height + 5
                    ax.plot([pos1, pos1, pos2, pos2], [height, height + 1, height + 1, height], 'k-', lw=1.5)
                    ax.text((pos1 + pos2) / 2, height + 1.5, star, ha='center', va='bottom', fontsize=12)
                break

def main():
    # Load data
    global df
    df = load_data('In vitro cytotoxicity assay.csv')

    groups = {
        'JNPs': ['Viability_np1', 'Viability_np2', 'Viability_np3'],
        'JNPs+AMF': ['Viability_np_amf1', 'Viability_np_amf2', 'Viability_np_amf3'],
        'JNPs+NIR': ['Viability_np_nir1', 'Viability_np_nir2', 'Viability_np_nir3'],
        'JNPs+PMT': ['Viability_np_amf_nir1', 'Viability_np_amf_nir2', 'Viability_np_amf_nir3'],
    }

    colors = {
        'JNPs': 'forestgreen',
        'JNPs+AMF': 'orchid',
        'JNPs+NIR': 'springgreen',
        'JNPs+PMT': 'crimson',
    }

    bar_width = 0.2
    x = np.arange(len(df))

    fig, ax = plt.subplots(figsize=(14, 7))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    plt.rcParams.update({'font.size': 18})

    group_positions, group_means = plot_bars_with_error(ax, x, groups, colors, bar_width)

    ax.set_xticks(x)
    ax.set_xticklabels(format_tick_labels(df['Nanoparticle concentration']), rotation=45, ha='right', fontweight='bold')

    ax.set_xlabel('Nanoparticle concentration (µg/ml)', fontsize=18, fontweight='bold')
    ax.set_ylabel('Cell viability (%)', fontsize=18, fontweight='bold')
    ax.tick_params(axis='x', width=2, labelsize=16, length=6)
    ax.tick_params(axis='y', width=2, labelsize=16, length=6)
    ax.set_yticklabels([str(int(tick)) if tick.is_integer() else str(tick) for tick in ax.get_yticks()], fontweight='bold')

    ax.grid(False)
    ax.legend(frameon=False, bbox_to_anchor=(0.8, 0.9), loc='upper left', borderaxespad=0., fontsize=16)

    print("One-way ANOVA across different concentrations for group: JNPs")
    np_data = df[groups['JNPs']]
    anova = stats.f_oneway(*[np_data.iloc[i].values for i in range(len(np_data))])
    print(f"ANOVA F-value: {anova.statistic:.4f}")
    print(f"ANOVA p-value: {anova.pvalue:.4e}")

    flattened_values = np_data.values.flatten()
    labels = np.repeat(df['Nanoparticle concentration'], 3)
    tukey = pairwise_tukeyhsd(flattened_values, labels)
    print("\nTukey HSD for different concentrations (JNPs group):")
    print(tukey)

    print("\nOne-way ANOVA across groups at each concentration:")
    adjacent_pairs = [('JNPs', 'JNPs+AMF'), ('JNPs+AMF', 'JNPs+NIR'), ('JNPs+NIR', 'JNPs+PMT')]

    # Pairs with known significance to force annotate
    forced_pairs = {
        1.5: [('JNPs', 'JNPs+PMT')],
        25.0: [('JNPs', 'JNPs+NIR')]
    }

    group_names = list(groups.keys())

    for idx, row in df.iterrows():
        concentration = row['Nanoparticle concentration']
        group_values = [row[groups[group]] for group in group_names]

        anova = stats.f_oneway(*group_values)
        print(f"\nConcentration: {concentration} µg/ml")
        print(f"ANOVA F-value: {anova.statistic:.4f}")
        print(f"ANOVA p-value: {anova.pvalue:.4e}")

        values = np.concatenate(group_values)
        group_labels = sum([[g]*3 for g in group_names], [])
        tukey = pairwise_tukeyhsd(values, group_labels)
        print("Tukey HSD between groups:")
        print(tukey)

        annotate_significance(ax, group_positions, group_means, tukey, adjacent_pairs, idx)

        # Check and annotate forced significant pairs
        if concentration in forced_pairs:
            annotate_significance(ax, group_positions, group_means, tukey, forced_pairs[concentration], idx)

    plt.tight_layout(rect=[0, 0, 0.85, 1])
    plt.savefig('In_vitro_cytotoxicity_assay_graph.tiff', dpi=600, bbox_inches='tight')
    plt.show()

if __name__ == '__main__':
    main()
